# Lisp Metacircula Interpreter

In [1]:
(require '[clojure.test :refer [is]])

nil

In [2]:
(declare $eval)

#'user/$eval

In [3]:
(import 'clojure.lang.IFn)

(defrecord Closure [env params body]
  IFn
  (applyTo [self args]
    (let [extended-env (merge env (zipmap params args))]
      ($eval body extended-env))))

(defn make-closure
  [env params body]
  (->Closure env params body))

#'user/make-closure

## Main Interpreter Function

### Special Forms in our Lisp Interpreter

- `QUOTE: (QUOTE datum)`
- `IF: (IF condition then-part else-part)`
- `LAMBDA: (LAMBDA (param1 param2 ...) body)`
- `LABEL: (LABEL name expr)`

In [4]:
(defn $eval-variable
  [expr env]
  (if (contains? env expr)
    (env expr)
    (throw (RuntimeException. 
            (str "Variable " expr " not found in current scope!")))))

#'user/$eval-variable

In [5]:
(defn $eval-list
  [expr env]
  (case (first expr)
    nil ()
    QUOTE (second expr)
    IF (let [[_ condition then-part else-part] expr] 
         (if ($eval condition env)
           ($eval then-part env)
           ($eval else-part env)))
    
    ; Ordinary function application
    (let [[fun & args] (map #($eval % env) expr)]
      (apply fun args))))

#'user/$eval-list

In [6]:
(defn $eval
  [expr env]
  (cond
    (symbol? expr) ($eval-variable expr env)
    (list? expr) ($eval-list expr env)
    :else expr))

#'user/$eval

In [7]:
;; Test that ordinary values evaluate to themselves.
(is (= 42 ($eval 42 {})))
(is (= true ($eval true {})))
(is (= "hello" ($eval "hello" {})))
(is (= nil ($eval nil {})))

true

In [8]:
;; Test that symbols represent bindings in the given scope (env).
(is (= 7 ($eval 'x '{x 7})))
(is (= 10 ($eval 'y '{x 7, y 10})))
(try
  ($eval 'z '{x 7, y 10})
  (is false)
  (catch RuntimeException e (is true)))

true

In [9]:
(is (= 'x ($eval '(QUOTE x) '{x 5})))
(is (= '(+ 2 3) ($eval '(QUOTE (+ 2 3)) {})))
(is (= 42 ($eval '(QUOTE 42) {})))

true

In [10]:
(is (= 2 ($eval '(IF 1 2 3) {})))
(is (= 3 ($eval '(IF nil 2 3) {})))
(is (= 10 ($eval '(IF x y z) '{x 5, y 10, z 20})))
(is (= 20 ($eval '(IF x y z) '{x false, y 10, z 20})))

true

In [11]:
(is (= 1 ($eval '(CAR (QUOTE (1 2 3))) {'CAR first})))
(is (= 'B ($eval '(CAR (CDR (QUOTE (A B C D E)))) {'CAR first, 'CDR rest})))
(is (= 16 ($eval '(+ (* 3 2) 5 (- 10 5)) {'+ +, '* *, '- -})))

true

In [12]:
(def c (make-closure {'+ +} '(x) '(+ x 1)))
(is (= {'+ +} (.env c)))
(is (= '(x) (.params c)))
(is (= '(+ x 1) (.body c)))
(is (= 6 (apply c '(5))))

true